In [ ]:
import pandas as pd

from selenium import webdriver  
from selenium.webdriver import ActionChains as AC
import chromedriver_autoinstaller
from tqdm import tqdm
from tqdm import tqdm_notebook
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [ ]:
keywords = ['방탄소년단','블랙핑크']  # 크롤링하고 싶은 아이돌 이름 추가
n = 10   # n : 조회수를 가져오고 싶은 영상 갯수

In [ ]:
for keyword in keywords:
    # 크롬창 띄우기
    chrome_path=chromedriver_autoinstaller.install()  # 윈도우 : chromedriver.exe
    driver=webdriver.Chrome(chrome_path)

    driver.get("https://www.youtube.com/results?search_query={}".format(keyword)) 
    time.sleep(2)
    
    # 필터 - 조회수 기준 정렬
    driver.find_element_by_link_text('필터').click()
    time.sleep(1)
    driver.find_element_by_link_text('조회수').click()
    time.sleep(1)
    
    # 영상 제목 크롤링
    overlays = "#video-title > yt-formatted-string"
    titles = driver.find_elements_by_css_selector(overlays)
    title_list = []
    for i in range(n):
        title = titles[i].text
        title_list.append(title)
        
    # 조회수 크롤링
    overlays = "#metadata-line > span:nth-child(1)"
    views = driver.find_elements_by_css_selector(overlays)
    view_list = []
    for i in range(n):
        view = views[i].text.split(' ')[1]
        if '만회' in view:
            view = int(view.replace('만회','0000'))
        elif '억회' in view:            
            view = view.replace('억회','')
            if '.' in view:
                view = int(view.replace('.',''))*10000000
            else:
                view = int(view)*100000000            
        
        view_list.append(view)
        
    # 판다스로 만들기
    df = pd.DataFrame({'title':title_list, 'view':view_list})
    
    # csv파일로 저장
    df.to_csv('youtube_view({}).csv'.format(keyword), encoding='utf-8-sig')